In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torchvision.transforms as T
import PIL
from PIL import Image
import numpy as np
import os
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [13]:
# memory check
t = torch.cuda.get_device_properties(0).total_memory
t/1024/1024/1024

5.99951171875

## Using Pix2Pix GAN architecture to translate black and white manga panels to colored versions

In [17]:
# https://github.com/phillipi/pix2pix/tree/master
# https://neptune.ai/blog/pix2pix-key-model-architecture-decisions

### Vanilla GAN

The generator takes in noise (Gaussian distribution) and generates samples similar to the original dataset that they are trained on.

In [20]:
class Generator(nn.Module):
    def __init__(self, zdim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.gen(x)

In [22]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid() # binary classification--yes or no
        )
    def forward(self, x):
            return self.disc(x)

### U-Net Generator for pix2pix cGAN

modified U-Net generator--encoder (downsample) and decoder (upsample)

encoder: [Convolution -> BatchNorm -> Leaky ReLU (helps gradients flow)]
decoder: [Transposed Convolution -> BatchNorm -> Dropout (first 3 blocks) -> ReLU]

notes: kind of like the semantic segmentation

In [25]:
# https://www.tensorflow.org/tutorials/generative/pix2pix